In [15]:
import math
from functools import reduce
from collections import deque
from heapq import (
    heapify, heappop, heappush
)


def pref_to_rank(pref):
    return {
        a: {b: idx for idx, b in enumerate(a_pref)}
        for a, a_pref in pref.items()
    }


def gale_shapley(A, B, A_pref, B_pref):
    """Create a stable matching using the
    Gale-Shapley algorithm.
    
    Args:
        A(set): customer list
        B(set): banker list
        A_pref(dict[str, list[str]]): customer preference list
        B_pref(dict[str, list[str]]): banker preference list

    Return: 
        list of (a, b) pairs.
    """
    B_rank = pref_to_rank(B_pref)
    ask_list = {a: deque(bs) for a, bs in A_pref.items()}
    pair = {}
    remaining_A = set(A)
    
    while len(remaining_A) > 0:
        a = remaining_A.pop()
        b = ask_list[a].popleft()
        if b not in pair:
            pair[b] = a
        else:
            a0 = pair[b]
            b_prefer_a0 = B_rank[b][a0] < B_rank[b][a]
            if b_prefer_a0:
                remaining_A.add(a)
            else:
                remaining_A.add(a0)
                pair[b] = a
    
    return [(a, b) for b, a in pair.items()]

In [31]:
def multiply_banker(bankers: list, banker_capacity: dict):
    return reduce(lambda x, y: x + y, [[b] * banker_capacity[b] for b in bankers])

In [41]:
def gale_shapley_1vn(custs, bankers, cust_pref, banker_pref, banker_capacity=None):
    """Create a stable matching using the Gale-Shapley algorithm and allowing for the capacity of each b

    Args:
        custs(set): customer list
        bankers(set): banker list
        cust_pref(dict[str, list[str]]): customer preference list
        banker_pref(dict[str, list[str]]): banker preference list
        banker_capacity(dict[str, int]): banker capacity, how

    Return:
        list of (cust, b) pairs.
    """
    if banker_capacity is None:
        split_evenly = math.ceil(len(custs) / len(bankers))
        banker_capacity = {b: split_evenly for b in bankers}
        
    B_rank = pref_to_rank(banker_pref)
    ask_list = {cust: deque(multiply_banker(bs, banker_capacity)) for cust, bs in cust_pref.items()}
    pair = {}
    remaining_cust = set(custs)

    while remaining_cust:
        cust = remaining_cust.pop()
        b = ask_list[cust].popleft()
        if b not in pair:
            pair[b] = [cust]
        elif len(pair[b]) < banker_capacity[b]:
                pair[b].append(cust)
        else:
            for cust0 in pair[b]:
                b_prefer_cust = B_rank[b][cust0] > B_rank[b][cust]
                if b_prefer_cust:
                    pair[b].remove(cust0)
                    pair[b].append(cust)
                    remaining_cust.add(cust0)
                    break
            else:
                remaining_cust.add(cust)

    return [(cust, b) for b, cust in pair.items()]

In [74]:
def gale_shapley_1vn_heap(custs, bankers, cust_pref, banker_pref, banker_capacity=None):
    """Create a stable matching using the Gale-Shapley algorithm and allowing for the capacity of each b

    Args:
        custs(set): customer list
        bankers(set): banker list
        cust_pref(dict[str, list[str]]): customer preference list
        banker_pref(dict[str, list[str]]): banker preference list
        banker_capacity(dict[str, int]): banker capacity, how

    Return:
        list of (cust, b) pairs.
    """
    if banker_capacity is None:
        split_evenly = math.ceil(len(custs) / len(bankers))
        banker_capacity = {b: split_evenly for b in bankers}
        
    B_rank = pref_to_rank(banker_pref)
    ask_list = {cust: deque(multiply_banker(bs, banker_capacity)) for cust, bs in cust_pref.items()}
    pair = {}
    remaining_cust = set(custs)

    while remaining_cust:
        cust = remaining_cust.pop()
        b = ask_list[cust].popleft()
        rank = -B_rank[b][cust]
        if b not in pair:
            pair[b] = [(rank, cust)]
            heapify(pair[b])
        elif len(pair[b]) < banker_capacity[b]:
            heappush(pair[b], (rank, cust))
        elif rank > pair[b][0][0]: # 比堆顶的客户排位高
            dumped = heappop(pair[b])[-1]
            heappush(pair[b], (rank, cust))
            remaining_cust.add(dumped)
        else:
            remaining_cust.add(cust)

    return [(cust, b) for b, cust in pair.items()]

In [71]:
custs = ['悟空', '白龙马', '沙僧', '八戒']
bankers = ['杨幂', 'hzj']
cust_pref = {
    cust: bankers[:] for cust in custs
}
cust_pref['沙僧'] = bankers[::-1]

banker_pref = {
    'hzj': custs[:],
    "杨幂": ['悟空', '沙僧', '白龙马', '八戒']
}

ans = gale_shapley_1vn(custs=custs, bankers=bankers, cust_pref=cust_pref, banker_pref=banker_pref)

In [72]:
ans

[(['悟空', '白龙马'], '杨幂'), (['沙僧', '八戒'], 'hzj')]

In [75]:
males = ['一夫', '汤焱', '小余', 'giegie', '小刘', '小宽']
females = ['郭郭', '斯宇', 'sq', '李总', '雨婷', '翟总']

male_pref = {
    "一夫": ['郭郭', 'sq', '雨婷', '斯宇', '翟总', '李总'],
    '汤焱': ['斯宇', 'sq', '郭郭', '雨婷', '李总', '翟总'],
    '小余': ['sq', '李总', '雨婷', '郭郭', '斯宇', '翟总'],
    'giegie': ['sq', '翟总', '李总', '郭郭', '斯宇', '雨婷'],
    '小刘': ['sq', '翟总', '雨婷', '李总', '郭郭', '斯宇'],
    '小宽': ['雨婷', 'sq', '李总', '斯宇', '郭郭', '翟总']
}

female_pref = {
    '郭郭': ['一夫', '小余', '小宽', '汤焱', '小刘', 'giegie'],
    '斯宇': ['汤焱', '小刘', '小宽', 'giegie', '一夫', '小余'],
    'sq': ['giegie', '小刘', '小余', '一夫', '汤焱', '小宽'],
    '李总': ['小余', '小宽', '小刘', 'giegie', '一夫', '汤焱'],
    '雨婷': ['一夫', '小宽','汤焱', '小刘', '小余', 'giegie'],
    '翟总': ['小刘', 'giegie', '小余', '一夫', '汤焱', '小宽'] # 喜欢猛男
}

males.reverse()
res = gale_shapley_1vn_heap(custs=males, bankers=females, cust_pref=male_pref, banker_pref=female_pref)

In [76]:
res

[([(0, 'giegie')], 'sq'),
 ([(0, '一夫')], '郭郭'),
 ([(0, '汤焱')], '斯宇'),
 ([(-1, '小宽')], '雨婷'),
 ([(0, '小余')], '李总'),
 ([(0, '小刘')], '翟总')]

In [83]:
employees = ['lishan', 'hzj', 'yiwei', 'jiajie', 'junru', 'yueping', 'like']
tasks = ['o', 't', 'a']

employee_pref = {
    'lishan': ['o', 'a', 't'],
    'hzj': ['a', 't', 'o'],
    'yiwei': ['o', 't', 'a'],
    'jiajie': ['t', 'o', 'a'],
    'junru': ['o', 't', 'a'],
    'yueping': ['a', 't', 'o'],
    'like': ['a', 't', 'o']
}

task_pref = {
    'o': ['hzj', 'lishan', 'yiwei', 'junru', 'like', 'jiajie', 'yueping'],
    't': ['jiajie', 'lishan', 'yueping','hzj', 'yiwei', 'like', 'junru'],
    'a': ['yueping', 'lishan', 'like', 'junru', 'hzj', 'jiajie', 'yiwei']
}

task_capacity = {
    'o': 3, 't': 2, 'a': 2
}


params = {"custs": employees, "cust_pref": employee_pref, "bankers": tasks, "banker_pref": task_pref}
gale_shapley_1vn_heap(banker_capacity=task_capacity, **params)

[([(-3, 'junru'), (-2, 'yiwei'), (-1, 'lishan')], 'o'),
 ([(-2, 'like'), (0, 'yueping')], 'a'),
 ([(-3, 'hzj'), (0, 'jiajie')], 't')]

In [84]:
gale_shapley_1vn(banker_capacity=task_capacity, **params)

[(['yiwei', 'junru', 'lishan'], 'o'),
 (['yueping', 'like'], 'a'),
 (['jiajie', 'hzj'], 't')]

In [ ]:
class Heap():
    def __init__(self, l: list):
        self.l = l
        self.len = len(l)
        
    def _push_down